# The Influence of Different Baselines on Program-Comprehension Studies
## Partially from replication package (Peitek et al.), indicated by "(from replication package)"

## Prerequisites

### Import libraries

In [ ]:
import mne
import numpy as np
import scipy.signal as signal
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import scipy.version
import sklearn
from matplotlib.patches import Circle, Rectangle
from scipy.integrate import simps
import scipy.stats as stats
from tqdm.notebook import tqdm
import os
import math
import itertools
from PIL import Image
import re
import shutil
from numpy import nan
from cliffs_delta import cliffs_delta

STYLES = [(0, (5, 1)),(0, (3, 1, 1, 1)),(0, (1, 1)),(0, (3, 1, 1, 1, 1, 1)),(0, ())]
COLORS = ["darkred","darkorange","darkcyan","green","y"]
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
matplotlib.rcParams['mathtext.default']='regular'

In [ ]:
print(f"Matplotlib: {matplotlib.__version__}")
print(f"Numpy: {np.__version__}")
print(f"Pandas: {pd.__version__}")
print(f"MNE: {mne.__version__}")
print(f"Scipy: {scipy.__version__}")
print(f"SKLearn: {sklearn.__version__}")

### Create folder to save result

In [ ]:
if not os.path.exists('./data/RQ'):
    os.makedirs('./data/RQ')

### Helper methods

In [ ]:
# Function to translate names used in data source to names used in paper
def translate(s):
    if s == "PC":
        return "ProgCompr"
    if s == "Text":
        return "Read"
    if s == "Rest":
        return "CrossFix"
    if s == "Matrix":
        return "ProbSolv"
    return s

# Function to convert a string seperated by whitespace characters back to python list (from replication package)
def string_to_list_string(data):
    data = data.replace(' ', ',')
    data = data.replace('\n', ',')
    data = ','.join([element for element in data.split(",") if len(element) > 0])
    if data[1] == ",":
        data = "[" + data[2:]
    return data

# (from replication package)
def get_spectrum(data, sampling_rate, method='welch', decibel=False, resolution='auto'):
    """
    Calculate amplitude or power spectrum

    data: Should be of shape (n_channels, n_samples)
    sampling_rate: Sampling rate... (float)
    method:
        * welch for power spectrum using Welch's method (recommended)
        * ft for simple Fourier transform (amplitude spectrum)
        * ps for power spectrum using a simple fourier transform
    decibel: Convert spectrum to decibel (bool)
    """

    axis = -1

    n_channels, n_samples = data.shape

    if resolution == 'auto':
        n_frequencies = n_samples
    elif isinstance(resolution, (int, float)):
        n_frequencies = np.round(sampling_rate / resolution).astype('int')
    else:
        raise ValueError('\'{}\''.format(resolution))

    # Spectrum
    if method in ['ft', 'ps']:
        # Using FFT
        # Get (complex) spectrum
        spec = np.fft.fft(data, n=n_frequencies, axis=axis)
        freq = np.fft.fftfreq(n_frequencies) * sampling_rate

        # Convert to real positive-sided spectrum
        spec = np.abs(spec)
        nyquist = 0.5 * sampling_rate
        is_positive = np.logical_or(np.logical_and(freq >= 0, freq <= nyquist), freq == -nyquist)
        n_pos = np.sum(is_positive)
        is_positive = np.repeat(is_positive[np.newaxis], n_channels, axis=0)
        spec = np.reshape(spec[is_positive], (n_channels, n_pos))
        freq = np.abs(freq[is_positive])
        is_double = np.logical_and(freq > 0, freq < nyquist)
        is_double = np.repeat(is_double[np.newaxis], n_channels, axis=0)
        spec[is_double] = 2 * spec[is_double]

        if method in ['ps']:
            # Get power spectral density
            spec = (1 / (sampling_rate * n_frequencies)) * spec ** 2

            # Convert to decibel if required
            if decibel:
                spec = _to_decibel(spec)
    elif method in ['welch', 'welch_db']:
        # Using Welch method
        freq, spec = signal.welch(data, sampling_rate, nperseg=n_frequencies, detrend='constant', axis=axis)

        # Convert to decibel if required
        if decibel:
            spec = _to_decibel(spec)
    else:
        raise RuntimeError('Unknown method \'{}\''.format(method))

    return spec, freq

# (from replication package)
def bandpower(spec, freq, freqband, relative=False):
    """
    Get band power within specified frequency band
    Alternatively: https://raphaelvallat.com/bandpower.html
    """

    spec = np.asarray(spec)
    freq = np.asarray(freq)
    freqband = np.asarray(freqband)

    if spec.ndim != 1:
        raise ValueError('Input \'spec\' bad: {}'.format(spec.shape))

    if freqband.ndim != 1 and freqband.shape[-1] != 2:
        raise ValueError('Input \'freqband\' bad: {}'.format(freqband.shape))

    # Frequency resolution
    step_freq = freq[1] - freq[0]

    # Find closest indices of band in frequency vector
    is_in_freqband = np.logical_and(freq >= np.min(freqband), freq <= np.max(freqband))

    # Integral approximation of the spectrum using Simpson's rule
    bp = simps(spec[is_in_freqband], dx=step_freq)

    if relative:
        bp = bp / simps(spec, dx=step_freq)

    return bp

# (from replication package)
def _to_decibel(spec):
    return 10 * np.log10(spec)


# Function to calculate mental work load from eeg data (from replication package)
def get_mental_work_load(eeg_path, sampling_rate=500):
    # read in eeg file
    eeg_data = mne.io.read_raw_fif(eeg_path, preload=True, verbose='ERROR')

    # get raw channel data and do mean average referencing
    eeg_data_raw = eeg_data.get_data()
    eeg_data_ref = eeg_data_raw - np.mean(eeg_data_raw, axis=0)

    # extract channel names of eeg_data
    channel_names = list(eeg_data.to_data_frame().columns[1:])

    # create mock events for cutting eeg data (number, len, id)
    # events are set every 50 entry to move the window 0.1 seconds every time
    events = np.array([(i, 0, 1) for i in range(0, eeg_data_ref.shape[-1], 50)])

    # create temporal eeg raw for cutting data into epochs
    tmp_raw = mne.io.RawArray(eeg_data_ref, eeg_data.info, verbose='ERROR')

    # create epochs which have a 3 second window and operate on event id 1
    epochs = mne.epochs.Epochs(tmp_raw, events, event_id=1, tmin=0, tmax=3, baseline=None, preload=True,
                               verbose='ERROR')

    # calculate Mental Workload for each window
    mwl_array = []

    for epoch_data_raw in epochs:
        # perform power spectrum analysis on eeg data
        spectrum, frequency = get_spectrum(epoch_data_raw, sampling_rate, method='welch', decibel=False,
                                           resolution='auto')

        # calculate Mental Workload by dividing the power spectrum of the relative theta band power divided by relative alpha band power
        # on channel Fz for theta and Pc for alpha
        theata_power = bandpower(spectrum[channel_names.index("Fz"), :], frequency, [4.0, 8.0], relative=True)
        alpha_power = bandpower(spectrum[channel_names.index("Pz"), :], frequency, [8.0, 13.0], relative=True)
        MWL = theata_power / alpha_power

        # append Mental Workload to array
        mwl_array.append(MWL)
    return mwl_array, len(tmp_raw), len(epochs)

## EEG

### Load data

In [ ]:
df_filtered = pd.read_csv(f"./data/filteredData/filtered_data.csv")
df_filtered

### Calculate Mental Workload for each participant for each algorithm

In [ ]:
# sampling rate of the EEG data
sampling_rate = 500

# create Mental Workload column
df_filtered["MentalWorkLoad_Program"] = [np.array([]) for i in range(len(df_filtered))]
df_filtered["MentalWorkLoad_Baseline"] = [np.array([]) for i in range(len(df_filtered))]
df_filtered["MentalWorkLoad_Program_Minus_Baseline"] = [np.array([]) for i in range(len(df_filtered))]
df_filtered["MentalWorkLoad_Seconds_Per_Sample"] = [np.array([]) for i in range(len(df_filtered))]

# iterate over each row anc calculate Mental Workload for the task
for idx in tqdm(range(len(df_filtered))):
    program_eeg_path = df_filtered.iloc[idx]["ProgramEEG"]
    baseline_eeg_path = df_filtered.iloc[idx]["BaselineEEG"]

    # calculate Mental Workload for each algorithm (from replication package)
    mwl_program, data_len, mwl_len = get_mental_work_load(program_eeg_path)

    # calculate Mental Workload for the baseline task (from replication package)
    mwl_baseline, data_len_b, mwl_len_b = get_mental_work_load(baseline_eeg_path)

    # calculate Mental Workload baseline corrected
    baseline_mean = np.mean(mwl_baseline)
    mwl_program_minus_baseline = mwl_program - baseline_mean

    # Calculate sample duration in seconds
    try:
        data_per_mwl_sample = data_len / mwl_len
        seconds_per_mwl_sample = data_per_mwl_sample / sampling_rate
    except:
        seconds_per_mwl_sample = None

    # append Mental Workload to the current row
    df_filtered.at[idx, "MentalWorkLoad_Program"] = mwl_program
    df_filtered.at[idx, "MentalWorkLoad_Baseline"] = mwl_baseline
    df_filtered.at[idx, "MentalWorkLoad_Program_Minus_Baseline"] = mwl_program_minus_baseline
    df_filtered.at[idx, "MentalWorkLoad_Seconds_Per_Sample"] = seconds_per_mwl_sample
df_filtered

### Calculate answer times

In [ ]:
df_filtered["BaselineAnswerTime"] = -1
df_filtered["ProgramAnswerTime"] = -1

# Iterate over all tasks
for idx in tqdm(range(len(df_filtered))):
    df_filtered.at[idx, "BaselineAnswerTime"] = df_filtered.iloc[idx]["BaselineEndTime"] - df_filtered.iloc[idx]["BaselineStartTime"]
    df_filtered.at[idx, "ProgramAnswerTime"] = df_filtered.iloc[idx]["ProgramEndTime"] - df_filtered.iloc[idx]["ProgramStartTime"]

df_filtered

### Checkpoint: Save calculated data (from replication package)

In [ ]:
# save the filtered dataframe to csv
df_tmp = df_filtered.copy()
df_tmp["MentalWorkLoad_Program"] = df_tmp["MentalWorkLoad_Program"].apply(lambda series: str(list(series)))
df_tmp["MentalWorkLoad_Baseline"] = df_tmp["MentalWorkLoad_Baseline"].apply(lambda series: str(list(series)))
df_tmp["MentalWorkLoad_Program_Minus_Baseline"] = df_tmp["MentalWorkLoad_Program_Minus_Baseline"].apply(lambda series: str(list(series)))
df_tmp.to_csv("./data/RQ/MentalWorkLoadRaw.csv", sep=";", index=False)

### Load saved data (from replication package)

In [ ]:
df_filtered = pd.read_csv("./data/RQ/MentalWorkLoadRaw.csv", sep=";")
df_filtered["MentalWorkLoad_Program"] = df_filtered["MentalWorkLoad_Program"].apply(string_to_list_string)
df_filtered["MentalWorkLoad_Baseline"] = df_filtered["MentalWorkLoad_Baseline"].apply(string_to_list_string)
df_filtered["MentalWorkLoad_Program_Minus_Baseline"] = df_filtered["MentalWorkLoad_Program_Minus_Baseline"].apply(string_to_list_string)
df_filtered["MentalWorkLoad_Program"] = df_filtered["MentalWorkLoad_Program"].apply(lambda x: np.array(eval(x)))
df_filtered["MentalWorkLoad_Baseline"] = df_filtered["MentalWorkLoad_Baseline"].apply(lambda x: np.array(eval(x)))
df_filtered["MentalWorkLoad_Program_Minus_Baseline"] = df_filtered["MentalWorkLoad_Program_Minus_Baseline"].apply(lambda x: np.array(eval(x)))
df_filtered

### Outlier removal

#### Drop MentalWorkLoad NaN
Because answer time was shorter than 3 seconds

In [ ]:
before = len(df_filtered)
df_filtered = df_filtered.drop(df_filtered[df_filtered["MentalWorkLoad_Program_Minus_Baseline"].apply(len).eq(0)].index)
df_filtered = df_filtered.drop(df_filtered[df_filtered["MentalWorkLoad_Baseline"].apply(len).eq(0)].index)
df_filtered = df_filtered.drop(df_filtered[df_filtered["MentalWorkLoad_Program"].apply(len).eq(0)].index)
df_filtered = df_filtered.reset_index(drop=True)
print("Dropped " + str(before-len(df_filtered)) + " rows")
df_filtered

#### Drop skipped

In [ ]:
before = len(df_filtered)
df_droped_because_skipped_baseline = df_filtered[df_filtered["BaselineSkipped"] == True].copy()
df_droped_because_skipped_program = df_filtered[df_filtered["ProgramSkipped"] == True].copy()
df_filtered = df_filtered.drop(df_filtered[df_filtered["BaselineSkipped"] == True].index)
df_filtered = df_filtered.drop(df_filtered[df_filtered["ProgramSkipped"] == True].index)
df_filtered = df_filtered.reset_index(drop=True)
print("Dropped " + str(before-len(df_filtered)) + " rows")
df_filtered

#### Save rows dropped because skipped

In [ ]:
df_droped_because_skipped_baseline.to_csv("./data/RQ/droped_because_skipped_baseline.csv", sep=";", index=False)
df_droped_because_skipped_baseline

In [ ]:
df_droped_because_skipped_program.to_csv("./data/RQ/droped_because_skipped_program.csv", sep=";", index=False)
df_droped_because_skipped_program

#### Drop outlier
Datapoints with answer time outside 1.5 times the interquartile range are dropped.

In [ ]:
# Create dataframe
df_task_quartiles = pd.DataFrame(columns = ["Task", "Q1", "Q2", "Q3", "Minimum", "Maximum"])
df_droped_because_outlier = pd.DataFrame(columns=df_filtered.columns)
df_droped_because_outlier["Cause"] = ""

before = len(df_filtered)

print("Calculate and drop outlier for algorithms")
for algorithm in tqdm(df_filtered["Algorithm"].unique()):
    df_algo = df_filtered[df_filtered["Algorithm"] == algorithm]

    # Calculate interquartile range for task
    q1, q2, q3 = np.percentile(df_algo["ProgramAnswerTime"], [25,50,75])
    minimum = q1 - 1.5*(q3-q1)
    maximum = q3 + 1.5*(q3-q1)

    # Save calculated range in dataframe
    df_task_quartiles = pd.concat([df_task_quartiles, pd.DataFrame(data={"Task": [algorithm], "Q1": [q1], "Q2": [q2], "Q3": [q3], "Minimum": [minimum], "Maximum": [maximum]})],ignore_index=True)

    for index, row in df_algo.iterrows():
        if (row["ProgramAnswerTime"] < minimum or row["ProgramAnswerTime"] > maximum):
            # Drop row, if it don't matches range
            df_droped_because_outlier.loc[len(df_droped_because_outlier)] = row.copy()
            df_droped_because_outlier.loc[len(df_droped_because_outlier)-1]["Cause"] = "PC"
            df_filtered = df_filtered.drop(index)

print("Calculate and drop outlier for baseline tasks")
for baseline_task in tqdm(df_filtered["BaselineTask"].unique()):
    df_baseline_task = df_filtered[df_filtered["BaselineTask"] == baseline_task]

    # Calculate interquartile range for task
    q1, q2, q3 = np.percentile(df_baseline_task["BaselineAnswerTime"], [25,50,75])
    minimum = q1 - 1.5*(q3-q1)
    maximum = q3 + 1.5*(q3-q1)

    # Save calculated range in dataframe
    df_task_quartiles = pd.concat([df_task_quartiles, pd.DataFrame(data={"Task": [baseline_task], "Q1": [q1], "Q2": [q2], "Q3": [q3], "Minimum": [minimum], "Maximum": [maximum]})],ignore_index=True)

    for index, row in df_baseline_task.iterrows():
        if (row["BaselineAnswerTime"] < minimum or row["BaselineAnswerTime"] > maximum):
            # Drop row, if it don't matches range
            df_droped_because_outlier.loc[len(df_droped_because_outlier)] = row.copy()
            df_droped_because_outlier.loc[len(df_droped_because_outlier)-1]["Cause"] = "Baseline"
            df_filtered = df_filtered.drop(index)

df_filtered = df_filtered.reset_index(drop=True)
print("Dropped " + str(before-len(df_filtered)) + " rows")

# Save quartiles and range for each task
df_task_quartiles.to_csv("./data/RQ/task_quartiles.csv", sep=";", index=False)
df_task_quartiles

#### Save rows dropped because outlier

In [ ]:
df_droped_because_outlier.to_csv("./data/RQ/droped_because_outlier.csv", sep=";", index=False)
df_droped_because_outlier

#### Save dataframe for creating topoplots
Subset of df_filtered with only the needed columns for topoplot creation

In [ ]:
if not os.path.exists("./data/RQ/topo_data"):
    os.makedirs("./data/RQ/topo_data")
df_topo = df_filtered[["Participant", "Algorithm", "Baseline", "ProgramEEG", "BaselineEEG"]]
df_topo.to_csv("./data/RQ/topo_data/data.csv", sep=";", index=False)
df_topo

### Calculate remaining rows per participant

In [ ]:
# Create dataframe
df_participant_rows = pd.DataFrame(columns = ["Participant", "Rows"])

# Add rows per participant
for participant in tqdm(df_filtered["Participant"].unique()):
    df_participant = df_filtered[df_filtered["Participant"] == participant]
    df_participant_rows = pd.concat([df_participant_rows, pd.DataFrame(data={"Participant": [participant], "Rows": [len(df_participant)]})],ignore_index=True)

# Sort and save dataframe
df_participant_rows.sort_values("Rows", inplace=True, ignore_index=True)
df_participant_rows.to_csv("./data/RQ/participant_rows.csv", sep=";", index=False)
df_participant_rows

**Note**
Run script until here, then exclude whole participants (<50%) and then run again, whithout them for further analysis. (Security feature, that no participant is "accidently" removed)

### Calculate metrics and stats for MWL (from replication package)

In [ ]:
# stats per algorithm per participant
filter = df_filtered.filter(
    ["MWLLength", "MWLMean", "MWLMedian", "MWLStd", "MWLMin", "MWLMax", "MWL_average_slope", "MWL_total_slope",
     "PeakToPeak", "MWL_max_slope"])
df_filtered = df_filtered.drop(filter, axis=1)

df_filtered.insert(loc=0, column="MWLLength", value=df_filtered["MentalWorkLoad_Program_Minus_Baseline"].apply(len))
df_filtered.insert(loc=0, column="MWLMean", value=df_filtered["MentalWorkLoad_Program_Minus_Baseline"].apply(np.mean))
df_filtered.insert(loc=0, column="MWL_Program_Mean", value=df_filtered["MentalWorkLoad_Program"].apply(np.mean))
df_filtered.insert(loc=0, column="MWL_Baseline_Mean", value=df_filtered["MentalWorkLoad_Baseline"].apply(np.mean))
df_filtered.insert(loc=0, column="MWLMedian", value=df_filtered["MentalWorkLoad_Program_Minus_Baseline"].apply(np.median))
df_filtered.insert(loc=0, column="MWLStd", value=df_filtered["MentalWorkLoad_Program_Minus_Baseline"].apply(np.std))
df_filtered.insert(loc=0, column="MWLMin", value=df_filtered["MentalWorkLoad_Program_Minus_Baseline"].apply(np.min))
df_filtered.insert(loc=0, column="MWLMax", value=df_filtered["MentalWorkLoad_Program_Minus_Baseline"].apply(np.max))
df_filtered.insert(loc=0, column="PeakToPeak", value=df_filtered["MentalWorkLoad_Program_Minus_Baseline"].apply(np.ptp))
df_filtered.insert(loc=0, column="MWL_max_slope",
                   value=df_filtered["MentalWorkLoad_Program_Minus_Baseline"].apply(lambda x: np.amax(np.diff(x))))
df_filtered.insert(loc=0, column="MWL_average_slope",
                   value=df_filtered["MentalWorkLoad_Program_Minus_Baseline"].apply(lambda x: np.mean(np.diff(x))))
df_filtered.insert(loc=0, column="MWL_total_slope",
                   value=df_filtered["MentalWorkLoad_Program_Minus_Baseline"].apply(lambda x: np.sum(np.diff(x))))

df_filtered

### Save metrics and stats for MWL to csv for further analysis

In [ ]:
df_filtered[["MWL_total_slope", "MWL_average_slope", "MWLMax", "MWLMin", "MWLStd", "MWLMedian", "MWLMean", "MWL_Program_Mean", "MWL_Baseline_Mean", "MWLLength",
             "Participant", "Algorithm"]].to_csv("./data/RQ/MentalWorkLoadStatsRaw.csv")

### Calculate answer time and correctness statistics per task

In [ ]:
# Create dataframe
df_task_statistics = pd.DataFrame(columns = ["Task", "AnswerTime_Minimum","AnswerTime_Maximum", "AnswerTime_Mean", "Correctness"])

for task in df_filtered["BaselineTask"].unique():
    df_task = df_filtered[df_filtered["BaselineTask"] == task]
    df_correct_task = df_task[df_task["BaselineCorrect"] == True]

    # Add to dataframe
    df_task_statistics = pd.concat([df_task_statistics, pd.DataFrame(data={"Task": task, "AnswerTime_Minimum": [df_task["BaselineAnswerTime"].min()], "AnswerTime_Maximum": [df_task["BaselineAnswerTime"].max()], "AnswerTime_Mean": [df_task["BaselineAnswerTime"].mean()], "Correctness": [len(df_correct_task)/len(df_task)]})],ignore_index=True)

for task in df_filtered["Algorithm"].unique():
    df_task = df_filtered[df_filtered["Algorithm"] == task]
    df_correct_task = df_task[df_task["ProgramCorrect"] == True]

    # Add to dataframe
    df_task_statistics = pd.concat([df_task_statistics, pd.DataFrame(data={"Task": task, "AnswerTime_Minimum": [df_task["ProgramAnswerTime"].min()], "AnswerTime_Maximum": [df_task["ProgramAnswerTime"].max()], "AnswerTime_Mean": [df_task["ProgramAnswerTime"].mean()], "Correctness": [len(df_correct_task)/len(df_task)]})],ignore_index=True)

df_task_statistics.to_csv("./data/RQ/task_statistics.csv", sep=";", index=False)
df_task_statistics

### Calculate length information for averaging mental work load

**Note**
For calculating the average we only take the sample slice from zero to the mean lenght of all datapoints of the same type (Program comprehension, Math, ...) into account. This means, that we have minimum 50% of the data for calculating an average.

In [ ]:
# Create dataframe
df_information = pd.DataFrame(columns = ["Type", "MedianLength","SampleCount", "MedianLength_PC", "SampleCount_PC", "75", "75_PC"])
length_pc_total = []

# Iterate over all baselines
for baseline in df_filtered["Baseline"].unique():
    df_baseline = df_filtered[df_filtered["Baseline"] == baseline]
    length = []
    length_pc = []

    # Add length for baseline and program comprehension
    for index, row in df_baseline.iterrows():
        length.append(len(row["MentalWorkLoad_Baseline"]))
        length_pc.append(len(row["MentalWorkLoad_Program"]))
        length_pc_total.append(len(row["MentalWorkLoad_Program"]))

    # Calculate median for baseline
    median_length = math.floor(np.median(length))

    # Count samples for each datapoint for baseline
    sample_count = [0] * median_length
    for l in length:
        for i in range(min(median_length, l)):
            sample_count[i] += 1

    # Calculate 75% marker for baseline
    marker_75_baseline = -1
    for i in range(len(sample_count)):
        if sample_count[i] / sample_count[0] >= 0.75:
            marker_75_baseline = i
    if marker_75_baseline == len(sample_count)-1:
        marker_75_baseline = -1

    # Calculate median for program comprehension
    median_length_pc = math.floor(np.median(length_pc))

    # Count samples for each datapoint for program comprehension
    sample_count_pc = [0] * median_length_pc
    for l in length_pc:
        for i in range(min(median_length_pc, l)):
            sample_count_pc[i] += 1

    # Calculate 75% marker for program comprehension
    marker_75_program = -1
    for i in range(len(sample_count_pc)):
        if sample_count_pc[i] / sample_count_pc[0] >= 0.75:
            marker_75_program = i
    if marker_75_program == len(sample_count_pc)-1:
        marker_75_program = -1

    # Add to dataframe
    df_information = pd.concat([df_information, pd.DataFrame(data={"Type": baseline, "MedianLength": median_length, "SampleCount": [sample_count], "MedianLength_PC": median_length_pc, "SampleCount_PC": [sample_count_pc], "75": [marker_75_baseline], "75_PC": [marker_75_program]})],ignore_index=True)

# Calculate median for program comprehension
median_length_pc_total = math.floor(np.median(length_pc_total))

# Count samples for each datapoint for program comprehension
sample_count_pc_total = [0] * median_length_pc_total
for l in length_pc_total:
    for i in range(min(median_length_pc_total, l)):
        sample_count_pc_total[i] += 1

# Calculate 75% marker for program comprehension
marker_75_program_total = -1
for i in range(len(sample_count_pc_total)):
    if sample_count_pc_total[i] / sample_count_pc_total[0] >= 0.75:
        marker_75_program_total = i
if marker_75_program_total == len(sample_count_pc_total)-1:
    marker_75_program_total = -1

# Add to dataframe
df_information = pd.concat([df_information, pd.DataFrame(data={"Type": "Program", "MedianLength": median_length_pc_total, "SampleCount": [sample_count_pc_total], "75": [marker_75_program_total]})],ignore_index=True)

seconds_per_sample_mean = df_filtered["MentalWorkLoad_Seconds_Per_Sample"].mean()
seconds_per_sample_round = round(seconds_per_sample_mean, 4)
samples_per_second = 1/seconds_per_sample_mean

df_information

### Calculate and plot average per Baseline

In [ ]:
# Create folder if not exists
if not os.path.exists("./data/RQ/plots/averaged_total"):
    os.makedirs("./data/RQ/plots/averaged_total")
if not os.path.exists("./data/RQ/plots/averaged_program_comprehension_with_baseline"):
    os.makedirs("./data/RQ/plots/averaged_program_comprehension_with_baseline")

# Create dataframe
df_averaged_over_all = pd.DataFrame(columns = ["Type", "MWL","MWL_Mean"])
df_averaged_program_comprehension_with_baseline = pd.DataFrame(columns = ["Baseline", "MWL","MWL_Minus_Baseline"])

# Init vars for program comprehension calculation
program_median_length = df_information[df_information['Type'] == "Program"]["MedianLength"].item()
program_sample_count = df_information[df_information['Type'] == "Program"]["SampleCount"].item()
program_marker_75 = df_information[df_information['Type'] == "Program"]["75"].item()
program_averaged = [0] * program_median_length

total_plot = []
total_baseline_plot = []
total_program_split_by_baseline_plot = []

total_detail_fig, total_detail_axs = plt.subplots(1, 4, sharex='all', sharey='all',figsize=(20,5))
count = 0
# Iterate over all baselines
baselines = df_filtered["Baseline"].unique()
for baseline in tqdm(baselines):
    # Init vars for baseline calculation
    df_baseline = df_filtered[df_filtered["Baseline"] == baseline]
    baseline_median_length = df_information[df_information['Type'] == baseline]["MedianLength"].item()
    baseline_sample_count = df_information[df_information['Type'] == baseline]["SampleCount"].item()
    baseline_marker_75 = df_information[df_information['Type'] == baseline]["75"].item()
    baseline_median_length_pc = df_information[df_information['Type'] == baseline]["MedianLength_PC"].item()
    baseline_sample_count_pc = df_information[df_information['Type'] == baseline]["SampleCount_PC"].item()
    baseline_marker_75_pc = df_information[df_information['Type'] == baseline]["75_PC"].item()
    baseline_averaged = [0] * baseline_median_length
    program_per_baseline_averaged = [0] * baseline_median_length_pc
    program_per_baseline_averaged_minus_baseline = [0] * baseline_median_length_pc

    for index, row in df_baseline.iterrows():
        mwl_base = row["MentalWorkLoad_Baseline"]
        mwl_program = row["MentalWorkLoad_Program"]
        mwl_program_minus_baseline = row["MentalWorkLoad_Program_Minus_Baseline"]
        # Add baseline mental work load
        for i in range(0, min(len(mwl_base), baseline_median_length)):
            baseline_averaged[i] += mwl_base[i]
        # Add program mental work load
        for i in range(0, min(len(mwl_program), program_median_length)):
            program_averaged[i] += mwl_program[i]
        # Add program mental work load with baseline
        for i in range(0, min(len(mwl_program), baseline_median_length_pc)):
            program_per_baseline_averaged[i] += mwl_program[i]
        # Add program minus baseline mental work load with baseline
        for i in range(0, min(len(mwl_program_minus_baseline), baseline_median_length_pc)):
            program_per_baseline_averaged_minus_baseline[i] += mwl_program_minus_baseline[i]

    # Average baseline mental work load
    for i in range(0, len(baseline_averaged)):
        baseline_averaged[i] = baseline_averaged[i] / baseline_sample_count[i]
    df_averaged_over_all = pd.concat([df_averaged_over_all, pd.DataFrame(data={"Type": translate(baseline), "MWL": [baseline_averaged], "MWL_Mean": np.mean(baseline_averaged)})],ignore_index=True)

    # Average program mental work load
    for i in range(0, len(program_per_baseline_averaged)):
        program_per_baseline_averaged[i] = program_per_baseline_averaged[i] / baseline_sample_count_pc[i]
        program_per_baseline_averaged_minus_baseline[i] = program_per_baseline_averaged_minus_baseline[i] / baseline_sample_count_pc[i]
    df_averaged_program_comprehension_with_baseline = pd.concat([df_averaged_program_comprehension_with_baseline, pd.DataFrame(data={"Baseline": translate(baseline), "MWL": [program_per_baseline_averaged], "MWL_Minus_Baseline": [program_per_baseline_averaged_minus_baseline]})],ignore_index=True)

    total_plot.append((program_per_baseline_averaged_minus_baseline, f"$ProgCompr_{{{translate(baseline)}}}$ > {translate(baseline)}"))
    total_baseline_plot.append((baseline_averaged, f"{translate(baseline)}"))
    total_program_split_by_baseline_plot.append((program_per_baseline_averaged, f"{translate(baseline)}"))

    # Plot averaged baseline mental work load and mean
    style_it=itertools.cycle(STYLES)
    color_it=itertools.cycle(COLORS)
    plt.figure()
    color=next(color_it)
    linestyle=next(style_it)
    p = plt.plot(baseline_averaged, color=color, linestyle=linestyle, label=f"{translate(baseline)}")
    plt.axhline(y=np.mean(baseline_averaged), color=color, linestyle=linestyle, label=f"{translate(baseline)} Mean")
    if baseline_marker_75 > 0:
        plt.axvline(x=baseline_marker_75, linestyle='solid', color='k')
        plt.text(baseline_marker_75,0.01,'75% ', ha='right', va='bottom', transform=plt.gca().get_xaxis_transform())
    plt.legend(handlelength=3)
    plt.xlabel(f"Seconds")
    plt.ylabel("Mental Load")
    plt.tight_layout()
    plt.grid(False)
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['right'].set_visible(False)

    # Convert samples in seconds
    locs, labels = plt.xticks()
    labels_seconds = []
    for label in labels:
        labels_seconds.append(str(round(float(re.sub(r'[^\x00-\x7F]+','-', label.get_text())) * seconds_per_sample_mean,2)))
    plt.xticks(locs[1:-1], labels_seconds[1:-1])

    plt.savefig("./data/RQ/plots/averaged_total/" + translate(baseline) + ".pdf", bbox_inches='tight', pad_inches=0)
    plt.close()

    # Plot averaged program mental work load and mean
    style_it=itertools.cycle(STYLES)
    color_it=itertools.cycle(COLORS)
    plt.figure()
    color=next(color_it)
    linestyle=next(style_it)
    plt.plot(program_per_baseline_averaged_minus_baseline, color=color, linestyle=linestyle, label=f"$ProgCompr_{{{translate(baseline)}}}$ > {translate(baseline)}")
    if baseline_marker_75_pc > 0:
        plt.axvline(x=baseline_marker_75_pc, linestyle='solid', color='k')
        plt.text(baseline_marker_75_pc,0.01,'75% ', ha='right', va='bottom', transform=plt.gca().get_xaxis_transform())
    plt.legend(handlelength=3)
    plt.xlabel(f"Seconds")
    plt.ylabel("Mental Load")
    plt.tight_layout()
    plt.grid(False)
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['right'].set_visible(False)

    # Convert samples in seconds
    locs, labels = plt.xticks()
    labels_seconds = []
    for label in labels:
        labels_seconds.append(str(round(float(re.sub(r'[^\x00-\x7F]+','-', label.get_text())) * seconds_per_sample_mean,2)))
    plt.xticks(locs[1:-1], labels_seconds[1:-1])

    plt.savefig("./data/RQ/plots/averaged_program_comprehension_with_baseline/" + translate(baseline) + ".pdf", bbox_inches='tight', pad_inches=0)
    plt.close()

    # Plot averaged program mental work load and mean with more detail
    style_it=itertools.cycle(STYLES)
    color_it=itertools.cycle(COLORS)
    plt.figure()
    color=next(color_it)
    linestyle=next(style_it)
    plt.plot(program_per_baseline_averaged, color=color, linestyle=linestyle, label=f"$ProgCompr_{{{translate(baseline)}}}$")
    color=next(color_it)
    linestyle=next(style_it)
    plt.plot(program_per_baseline_averaged_minus_baseline, color=color, linestyle=linestyle, label=f"$ProgCompr_{{{translate(baseline)}}}$ > {translate(baseline)}")
    plt.axhline(y=np.mean(baseline_averaged), color='r', label=f"{translate(baseline)} Mean")
    if baseline_marker_75_pc > 0:
        plt.axvline(x=baseline_marker_75_pc, linestyle='solid', color='k')
        plt.text(baseline_marker_75_pc,0.01,'75% ', ha='right', va='bottom', transform=plt.gca().get_xaxis_transform())
    plt.legend(handlelength=3)
    plt.xlabel(f"Seconds")
    plt.ylabel("Mental Load")
    plt.tight_layout()
    plt.grid(False)
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['right'].set_visible(False)

    # Convert samples in seconds
    locs, labels = plt.xticks()
    labels_seconds = []
    for label in labels:
        labels_seconds.append(str(round(float(re.sub(r'[^\x00-\x7F]+','-', label.get_text())) * seconds_per_sample_mean,2)))
    plt.xticks(locs[1:-1], labels_seconds[1:-1])

    plt.savefig("./data/RQ/plots/averaged_program_comprehension_with_baseline/" + translate(baseline) + "_detail.pdf", bbox_inches='tight', pad_inches=0)
    plt.close()

    # Plot averaged program mental work load and mean with more detail in subplot
    style_it=itertools.cycle(STYLES)
    color_it=itertools.cycle(COLORS)
    figure = total_detail_axs[count]
    color=next(color_it)
    linestyle=next(style_it)
    figure.plot(program_per_baseline_averaged, color=color, linestyle=linestyle, label=f"$ProgCompr_{{{translate(baseline)}}}$")
    color=next(color_it)
    linestyle=next(style_it)
    figure.plot(program_per_baseline_averaged_minus_baseline, color=color, linestyle=linestyle, label=f"$ProgCompr_{{{translate(baseline)}}}$ > {translate(baseline)}")
    figure.axhline(y=np.mean(baseline_averaged), color='r', label=f"{translate(baseline)} Mean")
    if baseline_marker_75_pc > 0:
        figure.axvline(x=baseline_marker_75_pc, linestyle='solid', color='k')
        figure.text(baseline_marker_75_pc,0.01,'75% ', ha='right', va='bottom', transform=figure.get_xaxis_transform())
    figure.set_title(translate(baseline))
    figure.legend(handlelength=3, loc=2)
    figure.set_xlabel(f"Samples")
    figure.set_ylabel("Mental Load")
    figure.grid(False)
    figure.spines['top'].set_visible(False)
    figure.spines['right'].set_visible(False)
    count += 1

plt.figure(total_detail_fig)
plt.tight_layout()
plt.savefig("./data/RQ/plots/averaged_program_comprehension_with_baseline/total_detail.pdf", bbox_inches='tight', pad_inches=0)
plt.close()

# Average program comprehension mental work load
for i in range(0, len(program_averaged)):
    program_averaged[i] = program_averaged[i] / program_sample_count[i]
df_averaged_over_all = pd.concat([df_averaged_over_all, pd.DataFrame(data={"Type": "Program", "MWL": [program_averaged], "MWL_Mean": np.mean(program_averaged)})],ignore_index=True)

# Plot averaged program comprehension mental work load and mean
style_it=itertools.cycle(STYLES)
color_it=itertools.cycle(COLORS)
plt.figure()
color=next(color_it)
linestyle=next(style_it)
p = plt.plot(program_averaged, color=color, linestyle=linestyle, label="ProgCompr")
plt.axhline(y=np.mean(program_averaged), color=color, linestyle=linestyle, label="ProgCompr Mean")
if program_marker_75 > 0:
        plt.axvline(x=program_marker_75, linestyle='solid', color='k')
        plt.text(program_marker_75,0.01,'75% ', ha='right', va='bottom', transform=plt.gca().get_xaxis_transform())
plt.legend(handlelength=3)
plt.xlabel(f"Seconds")
plt.ylabel("Mental Load")
plt.tight_layout()
plt.grid(False)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

# Convert samples in seconds
locs, labels = plt.xticks()
labels_seconds = []
for label in labels:
    labels_seconds.append(str(round(float(re.sub(r'[^\x00-\x7F]+','-', label.get_text())) * seconds_per_sample_mean,2)))
plt.xticks(locs[1:-1], labels_seconds[1:-1])

plt.savefig("./data/RQ/plots/averaged_total/program_comprehension.pdf", bbox_inches='tight', pad_inches=0)
plt.close()

# Plot averaged program mental work load and mean total
if len(total_plot) > 0:
    style_it=itertools.cycle(STYLES)
    color_it=itertools.cycle(COLORS)
    plt.figure(figsize=(6,3))
    plt.vlines(x=200,ymin=0.6,ymax=1.7, color='white')

for data in total_plot:
    color=next(color_it)
    linestyle=next(style_it)
    plt.plot(data[0], color=color, linestyle=linestyle, label=data[1])

if len(total_plot) > 0:
    plt.legend(handlelength=3)
    plt.xlabel(f"Seconds")
    plt.ylabel("Mental Load")
    plt.tight_layout()
    plt.grid(False)
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['right'].set_visible(False)

    # Convert samples in seconds
    locs, labels = plt.xticks()
    labels_seconds = []
    for label in labels:
        labels_seconds.append(str(round(float(re.sub(r'[^\x00-\x7F]+','-', label.get_text())) * seconds_per_sample_mean,2)))
    plt.xticks(locs[1:-1], labels_seconds[1:-1])

    plt.savefig("./data/RQ/plots/averaged_program_comprehension_with_baseline/total.pdf", bbox_inches='tight', pad_inches=0)
    plt.close()

# Plot averaged program mental work load and mean total
if len(total_plot) > 0:
    style_it=itertools.cycle(STYLES)
    color_it=itertools.cycle(COLORS)
    plt.figure()

for data in total_program_split_by_baseline_plot:
    color=next(color_it)
    linestyle=next(style_it)
    plt.plot(data[0], color=color, linestyle=linestyle, label=data[1])

if len(total_plot) > 0:
    plt.legend(handlelength=3)
    plt.xlabel(f"Seconds")
    plt.ylabel("Mental Load")
    plt.tight_layout()
    plt.grid(False)
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['right'].set_visible(False)

    # Convert samples in seconds
    locs, labels = plt.xticks()
    labels_seconds = []
    for label in labels:
        labels_seconds.append(str(round(float(re.sub(r'[^\x00-\x7F]+','-', label.get_text())) * seconds_per_sample_mean,2)))
    plt.xticks(locs[1:-1], labels_seconds[1:-1])

    plt.savefig("./data/RQ/plots/averaged_program_comprehension_with_baseline/total_program_split_by_baseline.pdf", bbox_inches='tight', pad_inches=0)
    plt.close()

# Plot averaged program mental work load and mean total
if len(total_baseline_plot) > 0:
    style_it=itertools.cycle(STYLES)
    color_it=itertools.cycle(COLORS)
    plt.figure(figsize=(6,3))
    plt.vlines(x=200,ymin=1.3,ymax=3, color='white')

for data in total_baseline_plot:
    color=next(color_it)
    linestyle=next(style_it)
    p = plt.plot(data[0], color=color, linestyle=linestyle, label=data[1])
    plt.axhline(y=np.mean(data[0]), color=color, linestyle=linestyle, label=f"_{data[1]} Mean", xmin=0.8)

if len(total_baseline_plot) > 0:
    color=next(color_it)
    linestyle=next(style_it)
    plt.plot(program_averaged, color=color, linestyle=linestyle, label="ProgCompr")
    plt.axhline(y=np.mean(program_averaged), color=color, linestyle=linestyle, label="_ProgCompr Mean", xmin=0.8)
    plt.text(0.9, 0.4, 'Mean', horizontalalignment='center', verticalalignment='center',transform=plt.gca().transAxes)
    plt.legend(handlelength=3, loc='upper left')
    plt.xlabel(f"Seconds")
    plt.ylabel("Mental Load")
    plt.tight_layout()
    plt.grid(False)
    plt.gca().spines['top'].set_visible(False)
    plt.gca().spines['right'].set_visible(False)

    # Convert samples in seconds
    locs, labels = plt.xticks()
    labels_seconds = []
    for label in labels:
        labels_seconds.append(str(round(float(re.sub(r'[^\x00-\x7F]+','-', label.get_text())) * seconds_per_sample_mean,2)))
    plt.xticks(locs[1:-1], labels_seconds[1:-1])

    plt.savefig("./data/RQ/plots/averaged_total/total.pdf", bbox_inches='tight', pad_inches=0)
    plt.close()

df_averaged_over_all.to_csv("./data/RQ/averaged_over_all.csv", sep=";", index=False)
df_averaged_program_comprehension_with_baseline.to_csv("./data/RQ/averaged_program_comprehension_with_baseline.csv", sep=";", index=False)
df_averaged_program_comprehension_with_baseline

### Statistic tests

#### Calculate data for statistic tests

In [ ]:
mwl_raw = {}
mwl_per_baseline = {}
mwl_minus_baseline_per_baseline = {}

mwl_program_total = []

# Iterate over all baselines
baselines = df_filtered["Baseline"].unique()
for baseline in tqdm(baselines):
    # Init vars for baseline calculation
    df_baseline = df_filtered[df_filtered["Baseline"] == baseline]

    raw = []
    mwl = []
    mwl_minus_baseline = []

    for index, row in df_baseline.iterrows():
        mwl_program_minus_baseline = row["MentalWorkLoad_Program_Minus_Baseline"]
        mwl_program = row["MentalWorkLoad_Program"]
        mwl_baseline = row["MentalWorkLoad_Baseline"]

        raw.append(np.mean(mwl_baseline))
        mwl.append(np.mean(mwl_program))
        mwl_program_total.append(np.mean(mwl_program))
        mwl_minus_baseline.append(np.mean(mwl_program_minus_baseline))

    mwl_raw[baseline] = raw
    mwl_per_baseline[baseline] = mwl
    mwl_minus_baseline_per_baseline[baseline] = mwl_minus_baseline

mwl_raw["ProgCompr"] = mwl_program_total

#### Shapiro

In [ ]:
# Create folder if not exists
if not os.path.exists("./data/RQ/statistics"):
    os.makedirs("./data/RQ/statistics")

# Create dataframe
df_shapiro = pd.DataFrame(columns = ["Baseline"])
df_shapiro_mb = pd.DataFrame(columns = ["Baseline"])
df_shapiro_raw = pd.DataFrame(columns = ["Data"])

# Iterate over all baselines
baselines = df_filtered["Baseline"].unique()
for baseline in baselines:
    shapiro = stats.shapiro(mwl_per_baseline[baseline])
    shapiro_mb = stats.shapiro(mwl_minus_baseline_per_baseline[baseline])
    shapiro_raw = stats.shapiro(mwl_raw[baseline])
    df_shapiro = pd.concat([df_shapiro, pd.DataFrame(data=[{"Baseline": translate(baseline), "Shapiro_Statistic": shapiro[0], "Shapiro_pValue": shapiro[1]}])],ignore_index=True)
    df_shapiro_mb = pd.concat([df_shapiro_mb, pd.DataFrame(data=[{"Baseline": translate(baseline), "Shapiro_Statistic": shapiro_mb[0], "Shapiro_pValue": shapiro_mb[1]}])],ignore_index=True)
    df_shapiro_raw = pd.concat([df_shapiro_raw, pd.DataFrame(data=[{"Data": translate(baseline), "Shapiro_Statistic": shapiro_raw[0], "Shapiro_pValue": shapiro_raw[1]}])],ignore_index=True)

shapiro_raw = stats.shapiro(mwl_raw["ProgCompr"])
df_shapiro_raw = pd.concat([df_shapiro_raw, pd.DataFrame(data=[{"Data": "ProgCompr", "Shapiro_Statistic": shapiro_raw[0], "Shapiro_pValue": shapiro_raw[1]}])],ignore_index=True)

# Save dataframe to file
df_shapiro.to_csv("./data/RQ/statistics/shapiro_no_baseline_correction.csv", sep=";", index=False)
df_shapiro_mb.to_csv("./data/RQ/statistics/shapiro_with_baseline_correction.csv", sep=";", index=False)
df_shapiro_raw.to_csv("./data/RQ/statistics/shapiro_raw.csv", sep=";", index=False)
pd.set_option("display.precision", 20)
df_shapiro

In [ ]:
pd.set_option("display.precision", 20)
df_shapiro_mb

In [ ]:
pd.set_option("display.precision", 20)
df_shapiro_raw

#### Mann Whitney U & Kruskal Wallis test

In [ ]:
# Create folder if not exists
if not os.path.exists("./data/RQ/statistics"):
    os.makedirs("./data/RQ/statistics")

# Create dataframe
df_significance_averaged_time = pd.DataFrame(columns = ["Baseline1", "Baseline2","Kruskal_Statistic","Kruskal_pValue", "MWU_Statistic", "MWU_pValue"])
df_significance_averaged_time_mb = pd.DataFrame(columns = ["Baseline1", "Baseline2","Kruskal_Statistic","Kruskal_pValue", "MWU_Statistic", "MWU_pValue"])
df_significance_averaged_time_raw = pd.DataFrame(columns = ["1", "2","Kruskal_Statistic","Kruskal_pValue", "MWU_Statistic", "MWU_pValue"])

# Iterate over all combinations of baselines
for combination in list(itertools.combinations(mwl_per_baseline.keys(), 2)):
    # Load mental work load data for first baseline of combination
    group1 = mwl_per_baseline[combination[0]]
    group1_mb = mwl_minus_baseline_per_baseline[combination[0]]
    group1_raw = mwl_raw[combination[0]]

    # Load mental work load data for second baseline of combination
    group2 = mwl_per_baseline[combination[1]]
    group2_mb = mwl_minus_baseline_per_baseline[combination[1]]
    group2_raw = mwl_raw[combination[1]]

    # Perform Mann Whitney U test
    mwu = stats.mannwhitneyu(group1, group2, alternative='two-sided')
    mwu_mb = stats.mannwhitneyu(group1_mb, group2_mb, alternative='two-sided')
    mwu_raw = stats.mannwhitneyu(group1_raw, group2_raw, alternative='two-sided')

    # Add to dataframe
    df_significance_averaged_time = pd.concat([df_significance_averaged_time, pd.DataFrame(data=[{"Baseline1": translate(combination[0]), "Baseline2": translate(combination[1]), "MWU_Statistic": mwu[0], "MWU_pValue": mwu[1]}])],ignore_index=True)
    df_significance_averaged_time_mb = pd.concat([df_significance_averaged_time_mb, pd.DataFrame(data=[{"Baseline1": translate(combination[0]), "Baseline2": translate(combination[1]), "MWU_Statistic": mwu_mb[0], "MWU_pValue": mwu_mb[1]}])],ignore_index=True)
    df_significance_averaged_time_raw = pd.concat([df_significance_averaged_time_raw, pd.DataFrame(data=[{"1": translate(combination[0]), "2": translate(combination[1]), "MWU_Statistic": mwu_raw[0], "MWU_pValue": mwu_raw[1]}])],ignore_index=True)

baselines = df_filtered["Baseline"].unique()
for baseline in baselines:
    group1_raw = mwl_raw["ProgCompr"]
    group2_raw = mwl_raw[baseline]

    # Perform Mann Whitney U test
    mwu_raw = stats.mannwhitneyu(group1_raw, group2_raw, alternative='two-sided')

    # Add to dataframe
    df_significance_averaged_time_raw = pd.concat([df_significance_averaged_time_raw, pd.DataFrame(data=[{"1": "ProgCompr", "2": translate(baseline), "MWU_Statistic": mwu_raw[0], "MWU_pValue": mwu_raw[1]}])],ignore_index=True)

# Iterate over all baselines
baselines = df_filtered["Baseline"].unique()
kruskal_data = []
kruskal_data_mb = []
kruskal_data_raw = []
for baseline in baselines:
    kruskal_data.append(mwl_per_baseline[baseline])
    kruskal_data_mb.append(mwl_minus_baseline_per_baseline[baseline])
    kruskal_data_raw.append(mwl_raw[baseline])

kruskal_data_raw.append(mwl_raw["ProgCompr"])

# Perform kruskal wallis test
kruskal = stats.kruskal(*kruskal_data)
kruskal_mb = stats.kruskal(*kruskal_data_mb)
kruskal_raw = stats.kruskal(*kruskal_data_raw)
# Add to dataframe
df_significance_averaged_time = pd.concat([df_significance_averaged_time, pd.DataFrame(data=[{"Kruskal_Statistic": kruskal[0], "Kruskal_pValue": kruskal[1]}])],ignore_index=True)
df_significance_averaged_time_mb = pd.concat([df_significance_averaged_time_mb, pd.DataFrame(data=[{"Kruskal_Statistic": kruskal_mb[0], "Kruskal_pValue": kruskal_mb[1]}])],ignore_index=True)
df_significance_averaged_time_raw = pd.concat([df_significance_averaged_time_raw, pd.DataFrame(data=[{"Kruskal_Statistic": kruskal_raw[0], "Kruskal_pValue": kruskal_raw[1]}])],ignore_index=True)

# Save dataframe to file
df_significance_averaged_time.to_csv("./data/RQ/statistics/significance_no_baseline_correction.csv", sep=";", index=False)
df_significance_averaged_time_mb.to_csv("./data/RQ/statistics/significance_with_baseline_correction.csv", sep=";", index=False)
df_significance_averaged_time_raw.to_csv("./data/RQ/statistics/significance_raw.csv", sep=";", index=False)
pd.set_option("display.precision", 20)
df_significance_averaged_time

In [ ]:
pd.set_option("display.precision", 20)
df_significance_averaged_time_mb

In [ ]:
pd.set_option("display.precision", 20)
df_significance_averaged_time_raw

#### Cliff's Delta

In [ ]:
# Create folder if not exists
if not os.path.exists("./data/RQ/statistics"):
    os.makedirs("./data/RQ/statistics")

# Create dataframe
df_cliffs = pd.DataFrame(columns = ["Baseline1", "Baseline2","D","Res"])
df_cliffs_mb = pd.DataFrame(columns = ["Baseline1", "Baseline2","D","Res"])
df_cliffs_raw = pd.DataFrame(columns = ["1", "2","D","Res"])

# Iterate over all combinations of baselines
for combination in list(itertools.combinations(mwl_per_baseline.keys(), 2)):
    # Load mental work load data for first baseline of combination
    group1 = mwl_per_baseline[combination[0]]
    group1_mb = mwl_minus_baseline_per_baseline[combination[0]]
    group1_raw = mwl_raw[combination[0]]

    # Load mental work load data for second baseline of combination
    group2 = mwl_per_baseline[combination[1]]
    group2_mb = mwl_minus_baseline_per_baseline[combination[1]]
    group2_raw = mwl_raw[combination[1]]

    # Perform Cliffs Delta
    d, res = cliffs_delta(group1, group2)
    d_mb, res_mb = cliffs_delta(group1_mb, group2_mb)
    d_raw, res_raw = cliffs_delta(group1_raw, group2_raw)

    # Add to dataframe
    df_cliffs = pd.concat([df_cliffs, pd.DataFrame(data=[{"Baseline1": translate(combination[0]), "Baseline2": translate(combination[1]), "D": d, "Res": res}])],ignore_index=True)

    # Add to dataframe
    df_cliffs_mb = pd.concat([df_cliffs_mb, pd.DataFrame(data=[{"Baseline1": translate(combination[0]), "Baseline2": translate(combination[1]), "D": d_mb, "Res": res_mb}])],ignore_index=True)

    # Add to dataframe
    df_cliffs_raw = pd.concat([df_cliffs_raw, pd.DataFrame(data=[{"1": translate(combination[0]), "2": translate(combination[1]), "D": d_raw, "Res": res_raw}])],ignore_index=True)

baselines = df_filtered["Baseline"].unique()
for baseline in baselines:
    group1_raw = mwl_raw["ProgCompr"]
    group2_raw = mwl_raw[baseline]

    # Perform Mann Whitney U test
    d_raw, res_raw = cliffs_delta(group1_raw, group2_raw)

    # Add to dataframe
    df_cliffs_raw = pd.concat([df_cliffs_raw, pd.DataFrame(data=[{"1": "ProgCompr", "2": translate(baseline), "D": d_raw, "Res": res_raw}])],ignore_index=True)

# Save dataframe to file
df_cliffs.to_csv("./data/RQ/statistics/cliffs_delta_no_baseline_correction.csv", sep=";", index=False)
df_cliffs_mb.to_csv("./data/RQ/statistics/cliffs_delta_with_baseline_correction.csv", sep=";", index=False)
df_cliffs_raw.to_csv("./data/RQ/statistics/cliffs_delta_raw.csv", sep=";", index=False)
pd.set_option("display.precision", 20)
df_cliffs

In [ ]:
pd.set_option("display.precision", 20)
df_cliffs_mb

In [ ]:
pd.set_option("display.precision", 20)
df_cliffs_raw

## Eye-tracking

### Import Fixation Data

In [ ]:
# Read in the fixation data
df_fixation = pd.read_csv('./data/filteredData/fixation_stats.csv', sep=";")

# Transform fixation strings to lists
df_fixation["Fixation_startT"] = df_fixation["Fixation_startT"].apply(string_to_list_string)
df_fixation["Fixation_endT"] = df_fixation["Fixation_endT"].apply(string_to_list_string)
df_fixation["Fixation_x"] = df_fixation["Fixation_x"].apply(string_to_list_string)
df_fixation["Fixation_y"] = df_fixation["Fixation_y"].apply(string_to_list_string)
df_fixation["Fixation_x_range"] = df_fixation["Fixation_x_range"].apply(string_to_list_string)
df_fixation["Fixation_y_range"] = df_fixation["Fixation_y_range"].apply(string_to_list_string)
df_fixation

### Calculate and plot scanpath for every task

In [ ]:
# Create folder if not exists
if not os.path.exists("./data/RQ/plots/scanpaths"):
    os.makedirs("./data/RQ/plots/scanpaths")

print("Generate scanpath for snippets")
snippets = df_fixation["Algorithm"].unique()
for snippet in tqdm(snippets):
    # Load snippet image
    image = Image.open(f"./data/Snippets/{snippet}.png")

    # Set snippet boundaries
    width, height = image.size
    x_low = int(1920 * 0.5) - int(width / 2)
    y_low = int(1080 * 0.5) - int(height / 2)
    x_high = int(1920 * 0.5) + int(width / 2)
    y_high = int(1080 * 0.5) + int(height / 2)

    # Get fixation data for snippet
    df_algo = df_fixation[(df_fixation["Algorithm"] == snippet) & (df_fixation["Type"] == "PC")]
    participants = df_algo["Participant"].unique()

    # Iterate over all participants
    for participant in participants:
        if len(df_algo[df_algo["Participant"] == participant]) != 1:
            raise ValueError("Participant has more than exactly one entry")

        # Get fixation data for participant
        df_algo_part = df_algo[df_algo["Participant"] == participant].reset_index().iloc[0]
        current_image = image.copy()
        fixation_start_array = np.array(eval(df_algo_part["Fixation_startT"]))
        fixation_end_array = np.array(eval(df_algo_part["Fixation_endT"]))
        fixation_x_coordinates = np.array(eval(df_algo_part["Fixation_x"]))
        fixation_y_coordinates = np.array(eval(df_algo_part["Fixation_y"]))
        fixations = np.stack((fixation_start_array, fixation_end_array, fixation_x_coordinates, fixation_y_coordinates), axis=1)

        # Set plot settings
        cm = plt.cm.get_cmap('inferno')
        cm = plt.cm.ScalarMappable(cmap=cm, norm=plt.Normalize(vmin=0, vmax=len(fixations)))
        patches = []

        # Calculate fixations
        for idx, (start, end, x, y) in enumerate(fixations):
            if x_low <= x <= x_high and y_low <= y <= y_high:
                x = int(x)
                y = int(y)
                x = x - x_low
                y = y - y_low
                color = cm.to_rgba(idx)
                patches.append(Circle((x, y), radius=10, color=color, alpha=0.5))

        # Plot fixations
        fig, ax = plt.subplots(1)
        ax.imshow(current_image)
        for p in patches:
            ax.add_patch(p)

        # Plot paths
        for idx in range(len(fixations) - 1):
            start_x = fixations[idx, 2] - x_low
            start_y = fixations[idx, 3] - y_low
            end_x = fixations[idx+1, 2] - x_low
            end_y = fixations[idx+1, 3] - y_low
            if 0 <= start_x <= width and 0 <= start_y <= height and 0 <= end_x <= width and 0 <= end_y <= height:
                color = cm.to_rgba(idx)
                ax.plot([start_x, end_x], [start_y, end_y], color=color, alpha=0.3)

        # Plot settings
        plt.xlim(0, width)
        plt.ylim(height, 0)
        plt.axis('off')
        plt.tight_layout()

        # Create folder if not exists
        if not os.path.exists(f"./data/RQ/plots/scanpaths/snippets/{snippet}"):
            os.makedirs(f"./data/RQ/plots/scanpaths/snippets/{snippet}")

        # Save plot
        plt.savefig(f"./data/RQ/plots/scanpaths/snippets/{snippet}/{participant}.pdf", bbox_inches='tight', pad_inches=0)
        plt.close()

print("Generate scanpath for baselines")
baseline_tasks = df_filtered["BaselineTask"].unique()
for baseline_task in tqdm(baseline_tasks):
    # Load snippet image
    image = Image.open(f"./data/BaselineSnippets/{baseline_task}.png")

    # Set snippet boundaries
    width, height = image.size
    x_low = int(1920 * 0.5) - int(width / 2)
    y_low = int(1080 * 0.5) - int(height / 2)
    x_high = int(1920 * 0.5) + int(width / 2)
    y_high = int(1080 * 0.5) + int(height / 2)

    # Get fixation data for snippet
    df_baseline = df_filtered[df_filtered["BaselineTask"] == baseline_task]
    participants = df_baseline["Participant"].unique()

    # Iterate over all participants
    for participant in participants:
        df_temp = df_filtered[(df_filtered["Participant"] == participant) & (df_filtered["BaselineTask"] == baseline_task)]["Algorithm"]
        for i in range(0, len(df_temp)):
            algorithm = df_temp.iloc[i]
            df_algo = df_fixation[(df_fixation["Algorithm"] == algorithm) & (df_fixation["Type"] == "Baseline")]
            if len(df_algo[df_algo["Participant"] == participant]) != 1:
                raise ValueError("Participant has more than exactly one entry")

            # Get fixation data for participant
            df_algo_part = df_algo[df_algo["Participant"] == participant].reset_index().iloc[0]
            current_image = image.copy()
            fixation_start_array = np.array(eval(df_algo_part["Fixation_startT"]))
            fixation_end_array = np.array(eval(df_algo_part["Fixation_endT"]))
            fixation_x_coordinates = np.array(eval(df_algo_part["Fixation_x"]))
            fixation_y_coordinates = np.array(eval(df_algo_part["Fixation_y"]))
            fixations = np.stack((fixation_start_array, fixation_end_array, fixation_x_coordinates, fixation_y_coordinates), axis=1)

            # Set plot settings
            cm = plt.cm.get_cmap('inferno')
            cm = plt.cm.ScalarMappable(cmap=cm, norm=plt.Normalize(vmin=0, vmax=len(fixations)))
            patches = []

            # Calculate fixations
            for idx, (start, end, x, y) in enumerate(fixations):
                if x_low <= x <= x_high and y_low <= y <= y_high:
                    x = int(x)
                    y = int(y)
                    x = x - x_low
                    y = y - y_low
                    color = cm.to_rgba(idx)
                    patches.append(Circle((x, y), radius=10, color=color, alpha=0.5))

            # Plot fixations
            fig, ax = plt.subplots(1)
            ax.imshow(current_image)
            for p in patches:
                ax.add_patch(p)

            # Plot paths
            for idx in range(len(fixations) - 1):
                start_x = fixations[idx, 2] - x_low
                start_y = fixations[idx, 3] - y_low
                end_x = fixations[idx+1, 2] - x_low
                end_y = fixations[idx+1, 3] - y_low
                if 0 <= start_x <= width and 0 <= start_y <= height and 0 <= end_x <= width and 0 <= end_y <= height:
                    color = cm.to_rgba(idx)
                    ax.plot([start_x, end_x], [start_y, end_y], color=color, alpha=0.3)

            # Plot settings
            plt.xlim(0, width)
            plt.ylim(height, 0)
            plt.axis('off')
            plt.tight_layout()

            # Create folder if not exists
            if not os.path.exists(f"./data/RQ/plots/scanpaths/baseline_tasks/{baseline_task}"):
                os.makedirs(f"./data/RQ/plots/scanpaths/baseline_tasks/{baseline_task}")

            # Save plot
            if baseline_task == "Rest_baseline":
                plt.savefig(f"./data/RQ/plots/scanpaths/baseline_tasks/{baseline_task}/{participant}_{i}.pdf", bbox_inches='tight', pad_inches=0)
            else:
                plt.savefig(f"./data/RQ/plots/scanpaths/baseline_tasks/{baseline_task}/{participant}.pdf", bbox_inches='tight', pad_inches=0)
            plt.close()

### Heatmaps

In [ ]:
# Create folder if not exists
if not os.path.exists("./data/RQ/plots/heatmaps"):
    os.makedirs("./data/RQ/plots/heatmaps")
if not os.path.exists("./data/RQ/plots/heatmaps/snippets"):
    os.makedirs("./data/RQ/plots/heatmaps/snippets")
if not os.path.exists("./data/RQ/plots/heatmaps/baseline_tasks"):
    os.makedirs("./data/RQ/plots/heatmaps/baseline_tasks")

print("Generate heatmaps for snippets")
# Iterate over all snippets
snippets = df_fixation["Algorithm"].unique()
for snippet in tqdm(snippets):
    # Load snippet image
    image = Image.open(f"./data/Snippets/{snippet}.png")

    # Set snippet boundaries
    width, height = image.size
    x_low = int(1920 * 0.5) - int(width / 2)
    y_low = int(1080 * 0.5) - int(height / 2)
    x_high = int(1920 * 0.5) + int(width / 2)
    y_high = int(1080 * 0.5) + int(height / 2)

    # Get fixation data for snippet
    df_algo = df_fixation[(df_fixation["Algorithm"] == snippet) & (df_fixation["Type"] == "PC")]
    participants = df_algo["Participant"].unique()

    heatmap_width = math.floor(1920/20)
    heatmap_height = math.floor(1080/20)
    snippet_data = [ [0]* heatmap_height for i in range(heatmap_width)]
    data_max = 0

    # Fill array for heatmap
    for index, row in df_algo.iterrows():
        fixation_start_array = np.array(eval(row["Fixation_startT"]))
        fixation_end_array = np.array(eval(row["Fixation_endT"]))
        fixation_x_coordinates = np.array(eval(row["Fixation_x"]))
        fixation_y_coordinates = np.array(eval(row["Fixation_y"]))
        fixations = np.stack((fixation_start_array, fixation_end_array, fixation_x_coordinates, fixation_y_coordinates), axis=1)

        for idx, (start, end, x, y) in enumerate(fixations):
            if x_low <= x <= x_high and y_low <= y <= y_high:
                x = int(x)
                y = int(y)
                x = x - x_low
                y = y - y_low
                snippet_data[math.floor(x/20)][math.floor(y/20)] += 1
                data_max = max(data_max, snippet_data[math.floor(x/20)][math.floor(y/20)])

    # Set plot settings
    cm = plt.cm.get_cmap('jet')
    cm = plt.cm.ScalarMappable(cmap=cm, norm=plt.Normalize(vmin=1, vmax=data_max))
    patches = []

    # Calculate fixations
    for x in range(0, heatmap_width):
        for y in range(0, heatmap_height):
            data = snippet_data[x][y]
            if data > 0:
                color = cm.to_rgba(data)
                patches.append(Rectangle((x*20, y*20), 19, 19, color=color, alpha=0.3))

    # Plot fixations
    fig, ax = plt.subplots(1)
    ax.imshow(image)
    for p in patches:
        ax.add_patch(p)

    # Plot settings
    plt.xlim(0, width)
    plt.ylim(height, 0)
    plt.axis('off')
    plt.tight_layout()

    # Save plot
    plt.savefig(f"./data/RQ/plots/heatmaps/snippets/{snippet}.pdf", bbox_inches='tight', pad_inches=0)
    plt.close()

print("Generate heatmaps for baseline tasks")
# Iterate over all baseline tasks
baseline_tasks = df_filtered["BaselineTask"].unique()
for baseline_task in tqdm(baseline_tasks):
    # Load snippet image
    image = Image.open(f"./data/BaselineSnippets/{baseline_task}.png")

    # Set snippet boundaries
    width, height = image.size
    x_low = int(1920 * 0.5) - int(width / 2)
    y_low = int(1080 * 0.5) - int(height / 2)
    x_high = int(1920 * 0.5) + int(width / 2)
    y_high = int(1080 * 0.5) + int(height / 2)

    heatmap_width = math.floor(1920/20)
    heatmap_height = math.floor(1080/20)
    snippet_data = [ [0]* heatmap_height for i in range(heatmap_width)]
    data_max = 0

    # Get fixation data for snippet
    df_baseline_task = df_filtered[(df_filtered["BaselineTask"] == baseline_task)]


    for index, row in df_baseline_task.iterrows():
        df_algo = df_fixation[(df_fixation["Algorithm"] == row["Algorithm"]) & (df_fixation["Participant"] == row["Participant"]) & (df_fixation["Type"] == "Baseline")]

        # Fill array for heatmap
        for index, row in df_algo.iterrows():
            fixation_start_array = np.array(eval(row["Fixation_startT"]))
            fixation_end_array = np.array(eval(row["Fixation_endT"]))
            fixation_x_coordinates = np.array(eval(row["Fixation_x"]))
            fixation_y_coordinates = np.array(eval(row["Fixation_y"]))
            fixations = np.stack((fixation_start_array, fixation_end_array, fixation_x_coordinates, fixation_y_coordinates), axis=1)

            for idx, (start, end, x, y) in enumerate(fixations):
                if x_low <= x <= x_high and y_low <= y <= y_high:
                    x = int(x)
                    y = int(y)
                    x = x - x_low
                    y = y - y_low
                    snippet_data[math.floor(x/20)][math.floor(y/20)] += 1
                    data_max = max(data_max, snippet_data[math.floor(x/20)][math.floor(y/20)])

    # Set plot settings
    cm = plt.cm.get_cmap('jet')
    cm = plt.cm.ScalarMappable(cmap=cm, norm=plt.Normalize(vmin=1, vmax=data_max))
    patches = []

    # Calculate fixations
    for x in range(0, heatmap_width):
        for y in range(0, heatmap_height):
            data = snippet_data[x][y]
            if data > 0:
                color = cm.to_rgba(data)
                patches.append(Rectangle((x*20, y*20), 19, 19, color=color, alpha=0.4))

    # Plot fixations
    fig, ax = plt.subplots(1)
    ax.imshow(image)
    for p in patches:
        ax.add_patch(p)

    # Plot settings
    plt.xlim(0, width)
    plt.ylim(height, 0)
    plt.axis('off')
    plt.tight_layout()

    # Save plot
    plt.savefig(f"./data/RQ/plots/heatmaps/baseline_tasks/{baseline_task}.pdf", bbox_inches='tight', pad_inches=0)
    plt.close()

### Topoplots

#### Load data

In [ ]:
df_topo = pd.read_csv("./data/RQ/topo_data/data.csv", sep=";")

channel_len = 751

columns = []
for i in range(1,channel_len+1):
    columns.append(str(i))

columns_with_channel = ["Channel"]
columns_with_channel.extend(columns)

df_topo

#### Generate eeg data for topoplots

In [ ]:
# Create folder if not exists
if not os.path.exists("./data/RQ/topo_data/raw/eeg"):
    os.makedirs("./data/RQ/topo_data/raw/eeg")

# Read in montage and calculate the electrode positions
montage_path = "./data/EEG/AC-64.bvef"
montage = mne.channels.read_custom_montage(montage_path, head_size=0.085)

for index, row in tqdm(df_topo.iterrows(), total=df_topo.shape[0]):
    participant = row["Participant"]
    snippet = row["Algorithm"]

    # Get the path for the baseline eeg data
    baseline_eeg_path = row["BaselineEEG"]

    # Read in the baseline eeg data
    eeg_data_baseline = mne.io.read_raw_fif(baseline_eeg_path, verbose='ERROR')

    # Get raw channel data and do mean average referencing
    eeg_data_baseline_raw = eeg_data_baseline.get_data()
    eeg_data_baseline_ref = eeg_data_baseline_raw - np.mean(eeg_data_baseline_raw, axis=0)

    # Set montage
    eeg_data_baseline.set_montage(montage, verbose='ERROR')

    # Save baseline eeg data for topo
    eeg_data_baseline_topo = mne.io.RawArray(eeg_data_baseline_ref, eeg_data_baseline.info, verbose='ERROR')
    eeg_data_baseline_topo.save(f"./data/RQ/topo_data/raw/eeg/{participant}_{snippet}_baseline_raw.fif", verbose='ERROR')

    # Get the path for the program eeg data
    program_eeg_path = row["ProgramEEG"]

    # Read in the program eeg data
    eeg_data_program = mne.io.read_raw_fif(program_eeg_path, verbose='ERROR')

    # Get raw channel data and do mean average referencing
    eeg_data_program_raw = eeg_data_program.get_data()
    eeg_data_program_ref = eeg_data_program_raw - np.mean(eeg_data_program_raw, axis=0)

    # Set montage
    eeg_data_program.set_montage(montage, verbose='ERROR')

    # Save program eeg data for topo
    eeg_data_progra_topo = mne.io.RawArray(eeg_data_program_ref, eeg_data_program.info, verbose='ERROR')
    eeg_data_progra_topo.save(f"./data/RQ/topo_data/raw/eeg/{participant}_{snippet}_program_raw.fif", verbose='ERROR')

#### Calculate PSDS

In [ ]:
# Create folder if not exists
if not os.path.exists("./data/RQ/topo_data/raw/psds"):
    os.makedirs("./data/RQ/topo_data/raw/psds")

# Read montage data
montage_path = "./data/EEG/AC-64.bvef"
montage = mne.channels.read_custom_montage(montage_path, head_size=0.085)

for index, row in tqdm(df_topo.iterrows(), total=df_topo.shape[0]):
    participant = row["Participant"]
    snippet = row["Algorithm"]

    # Read baseline eeg data
    eeg_data_baseline = mne.io.read_raw_fif(f"./data/RQ/topo_data/raw/eeg/{participant}_{snippet}_baseline_raw.fif", verbose='ERROR')
    eeg_data_baseline_array = mne.io.RawArray(eeg_data_baseline.get_data(), eeg_data_baseline.info, verbose='ERROR')

    # Generate epochs
    events_baseline = mne.make_fixed_length_events(eeg_data_baseline_array, id=1, duration=0.1)
    epochs_baseline = mne.epochs.Epochs(eeg_data_baseline_array, events_baseline, event_id=1, tmin=0, tmax=3, baseline=None, preload=True, verbose='ERROR')

    # Get channel data and scalings
    ch_type = mne.channels._get_ch_type(epochs_baseline, None)
    scalings = mne.defaults._handle_default('scalings', None)
    scaling = scalings[ch_type]

    # Generate psd
    psd_baseline, freq_baseline = mne.time_frequency.psd_multitaper(epochs_baseline, verbose='ERROR')
    psd_baseline_average = psd_baseline.mean(axis=0)
    psd_baseline_average *= scaling**2

    # Convert values to decibel
    for x in range(0, psd_baseline_average.shape[0]):
        for y in range(0, psd_baseline_average.shape[1]):
            psd_baseline_average[x,y] = _to_decibel(psd_baseline_average[x,y])

    # Save psd data
    np.savetxt(f"./data/RQ/topo_data/raw/psds/{participant}_{snippet}_baseline.csv", psd_baseline_average, delimiter=";")

    # Free memory
    del epochs_baseline
    del events_baseline
    del eeg_data_baseline_array
    del eeg_data_baseline

    # Read program eeg data
    eeg_data_program = mne.io.read_raw_fif(f"./data/RQ/topo_data/raw/eeg/{participant}_{snippet}_program_raw.fif", verbose='ERROR')
    eeg_data_program_array = mne.io.RawArray(eeg_data_program.get_data(), eeg_data_program.info, verbose='ERROR')

    # Generate epochs
    events_program = mne.make_fixed_length_events(eeg_data_program_array, id=1, duration=0.1)
    epochs_program = mne.epochs.Epochs(eeg_data_program_array, events_program, event_id=1, tmin=0, tmax=3, baseline=None, preload=True, verbose='ERROR')

    # Get channel data and scalings
    ch_type = mne.channels._get_ch_type(epochs_program, None)
    scalings = mne.defaults._handle_default('scalings', None)
    scaling = scalings[ch_type]

    # Generate psd
    psd_program, freq_program = mne.time_frequency.psd_multitaper(epochs_program, verbose='ERROR')
    psd_program_average = psd_program.mean(axis=0)
    psd_program_average *= scaling**2

    # Convert values to decibel
    for x in range(0, psd_program_average.shape[0]):
        for y in range(0, psd_program_average.shape[1]):
            psd_program_average[x,y] = _to_decibel(psd_program_average[x,y])

    # Save psd data
    np.savetxt(f"./data/RQ/topo_data/raw/psds/{participant}_{snippet}_program.csv", psd_program_average, delimiter=";")

    # Save info data and freq data for later use (equally for all tasks)
    if not os.path.exists("./data/RQ/topo_data/info-epo.fif"):
        epochs_program.save(f"./data/RQ/topo_data/info-epo.fif", overwrite=True)
    if not os.path.exists("./data/RQ/topo_data/freq.csv"):
        np.savetxt(f"./data/RQ/topo_data/freq.csv", freq_program, delimiter=";")

    # Free memory
    del epochs_program
    del events_program
    del eeg_data_program_array
    del eeg_data_program

    # Calculate psd for program comprehension minus baseline data
    psd_program_minus_baseline_average = np.subtract(psd_program_average, psd_baseline_average)

    # Save psd data
    np.savetxt(f"./data/RQ/topo_data/raw/psds/{participant}_{snippet}_program_minus_baseline.csv", psd_program_minus_baseline_average, delimiter=";")

#### Merge PSDS per baseline per participant

In [ ]:
# Create folder if not exists
if not os.path.exists("./data/RQ/topo_data/raw_participant"):
    os.makedirs("./data/RQ/topo_data/raw_participant")
# Create folder if not exists
if not os.path.exists("./data/RQ/topo_data/raw_all"):
    os.makedirs("./data/RQ/topo_data/raw_all")

for baseline in tqdm(df_topo["Baseline"].unique()):
    df_baseline = df_topo[df_topo["Baseline"] == baseline]

    for participant in tqdm(df_baseline["Participant"].unique()):
        df_participant = df_baseline[df_baseline["Participant"] == participant]

        # Init dataframes for participant psd data
        df_baseline_data_participant = pd.DataFrame(columns=columns_with_channel)
        df_program_data_participant = pd.DataFrame(columns=columns_with_channel)
        df_program_minus_baseline_data_participant = pd.DataFrame(columns=columns_with_channel)

        for index, row in df_participant.iterrows():
            snippet = row["Algorithm"]

            # Load psd data
            baseline_psds = np.loadtxt(f"./data/RQ/topo_data/raw/psds/{participant}_{snippet}_baseline.csv", delimiter=";")
            program_psds = np.loadtxt(f"./data/RQ/topo_data/raw/psds/{participant}_{snippet}_program.csv", delimiter=";")
            program_minus_baseline_psds = np.loadtxt(f"./data/RQ/topo_data/raw/psds/{participant}_{snippet}_program_minus_baseline.csv", delimiter=";")

            channel_number = 1
            for channel in baseline_psds:
                # Concat channel number and baseline psd data for channel
                baseline_data = [channel_number]
                baseline_data.extend(channel)

                # Add channel psd data to dataframes
                df_baseline_data_participant = pd.concat([df_baseline_data_participant, pd.DataFrame(data=[baseline_data], columns=columns_with_channel)],ignore_index=True)
                channel_number += 1

            channel_number = 1
            for channel in program_psds:
                # Concat channel number and program psd data for channel
                program_data = [channel_number]
                program_data.extend(channel)

                # Add channel psd data to dataframes
                df_program_data_participant = pd.concat([df_program_data_participant, pd.DataFrame(data=[program_data], columns=columns_with_channel)],ignore_index=True)
                channel_number += 1

            channel_number = 1
            for channel in program_minus_baseline_psds:
                # Concat channel number and program psd data for channel
                program_minus_baseline_data = [channel_number]
                program_minus_baseline_data.extend(channel)

                # Add channel psd data to dataframes
                df_program_minus_baseline_data_participant = pd.concat([df_program_minus_baseline_data_participant, pd.DataFrame(data=[program_minus_baseline_data], columns=columns_with_channel)],ignore_index=True)
                channel_number += 1

        # Save baseline psd data for participant
        df_baseline_data_participant = df_baseline_data_participant.groupby('Channel')[columns_with_channel[1:]].agg('mean')
        baseline_data_participant = df_baseline_data_participant.to_numpy()
        np.savetxt(f"./data/RQ/topo_data/raw_participant/{participant}_{baseline}_baseline.csv", baseline_data_participant, delimiter=";")

        # Save program psd data for participant
        df_program_data_participant = df_program_data_participant.groupby('Channel')[columns_with_channel[1:]].agg('mean')
        program_data_participant = df_program_data_participant.to_numpy()
        np.savetxt(f"./data/RQ/topo_data/raw_participant/{participant}_{baseline}_program.csv", program_data_participant, delimiter=";")

        # Save program minus baseline psd data for participant
        df_program_minus_baseline_data_participant = df_program_minus_baseline_data_participant.groupby('Channel')[columns_with_channel[1:]].agg('mean')
        program_minus_baseline_data_participant = df_program_minus_baseline_data_participant.to_numpy()
        np.savetxt(f"./data/RQ/topo_data/raw_participant/{participant}_{baseline}_program_minus_baseline.csv", program_minus_baseline_data_participant, delimiter=";")

#### PSDS Outlier detection

In [ ]:
# Create folder if not exists
if not os.path.exists("./data/RQ/psds"):
    os.makedirs("./data/RQ/psds")
if not os.path.exists("./data/RQ/psds/participant"):
    os.makedirs("./data/RQ/psds/participant")

df_dropped_channel = pd.DataFrame(columns=["Participant", "Baseline", "Type", "Channel"])

for participant in tqdm(df_topo["Participant"].unique()):
    df_participant = df_topo[df_topo["Participant"] == participant]

    if participant != 5:
        continue

    for baseline in df_participant["Baseline"].unique():
        df_baseline = df_participant[df_participant["Baseline"] == baseline]

        # Init dataframes for participant psd data
        df_baseline_data_participant = pd.read_csv(f"./data/RQ/topo_data/raw_participant/{participant}_{baseline}_baseline.csv", delimiter=";", names=columns)
        df_baseline_data_participant = df_baseline_data_participant.set_index(np.arange(1,65,1))
        df_program_data_participant = pd.read_csv(f"./data/RQ/topo_data/raw_participant/{participant}_{baseline}_program.csv", delimiter=";", names=columns)
        df_program_data_participant = df_program_data_participant.set_index(np.arange(1,65,1))

        df_psd_data_baseline = pd.DataFrame(columns=["Channel", "PSD"])
        df_psd_data_program = pd.DataFrame(columns=["Channel", "PSD"])

        all_psd_baseline = []
        all_psd_program = []

        for row_index, row in df_baseline_data_participant.iterrows():
            for col_index, value in row.iteritems():
                df_psd_data_baseline = pd.concat([df_psd_data_baseline, pd.DataFrame(data=[{"Channel": row_index, "PSD": value}])],ignore_index=True)
                all_psd_baseline.append(value)

        for row_index, row in df_program_data_participant.iterrows():
            for col_index, value in row.iteritems():
                df_psd_data_program = pd.concat([df_psd_data_program, pd.DataFrame(data=[{"Channel": row_index, "PSD": value}])],ignore_index=True)
                all_psd_program.append(value)

        # Calculate interquartile range for task
        q1_baseline, q2_baseline, q3_baseline = np.percentile(all_psd_baseline, [25,50,75])
        minimum_baseline = q1_baseline - 1.5*(q3_baseline-q1_baseline)
        maximum_baseline = q3_baseline + 1.5*(q3_baseline-q1_baseline)

        print(baseline)
        print("Baseline")
        print(f"Q1: {q1_baseline}")
        print(f"Q3: {q3_baseline}")
        print(f"Mean: {np.mean(all_psd_baseline)}")
        print(f"Max: {np.max(all_psd_baseline)}")

        q1_program, q2_program, q3_program = np.percentile(all_psd_program, [25,50,75])
        minimum_program = q1_program - 1.5*(q3_program-q1_program)
        maximum_program = q3_program + 1.5*(q3_program-q1_program)

        print("Program")
        print(f"Q1: {q1_program}")
        print(f"Q3: {q3_program}")
        print(f"Mean: {np.mean(all_psd_program)}")
        print(f"Max: {np.max(all_psd_program)}")

        outlier_per_channel_baseline = dict(zip(np.arange(1,65,1),np.zeros(64)))
        outlier_per_channel_program = dict(zip(np.arange(1,65,1),np.zeros(64)))

        for index, row in df_psd_data_baseline.iterrows():
            if row["PSD"] < minimum_baseline or row["PSD"] > maximum_baseline:
                outlier_per_channel_baseline[row["Channel"]] += 1

        df_outlier_per_channel_baseline_save = pd.DataFrame(data=outlier_per_channel_baseline, index=[0])
        df_outlier_per_channel_baseline_save.to_csv(f"./data/RQ/psds/participant/{participant}_{baseline}_outlier_per_channel_baseline.csv", sep=";")

        for index, row in df_psd_data_program.iterrows():
            if row["PSD"] < minimum_program or row["PSD"] > maximum_program:
                outlier_per_channel_program[row["Channel"]] += 1

        df_outlier_per_channel_program_save = pd.DataFrame(data=outlier_per_channel_program, index=[0])
        df_outlier_per_channel_program_save.to_csv(f"./data/RQ/psds/participant/{participant}_{baseline}_outlier_per_channel_program.csv", sep=";")

        for key, value in outlier_per_channel_baseline.items():
            if value > channel_len/2:
                df_dropped_channel = pd.concat([df_dropped_channel, pd.DataFrame(data=[{"Participant": participant, "Baseline": baseline ,"Type" : "Baseline", "Channel": key}])],ignore_index=True)

        for key, value in outlier_per_channel_program.items():
            if value > channel_len/2:
                df_dropped_channel = pd.concat([df_dropped_channel, pd.DataFrame(data=[{"Participant": participant, "Baseline": baseline, "Type" : "Program", "Channel": key}])],ignore_index=True)

df_dropped_channel.drop_duplicates(inplace=True, ignore_index=True)

# Save dataframe to file
df_dropped_channel.to_csv("./data/RQ/psds/dropped_channel.csv", sep=";", index=False)
df_dropped_channel

#### Drop PSD outlier

In [ ]:
if not os.path.exists("./data/RQ/topo_data/filtered_participant"):
    os.makedirs("./data/RQ/topo_data/filtered_participant")

# Copy files to filtered data
raw_participant_folder = "./data/RQ/topo_data/raw_participant/"
filtered_participant_folder = "./data/RQ/topo_data/filtered_participant/"

# fetch all files
for file_name in os.listdir(raw_participant_folder):
    # construct full file path
    source = raw_participant_folder + file_name
    destination = filtered_participant_folder + file_name
    # copy only files
    if os.path.isfile(source):
        shutil.copy(source, destination)

for participant in tqdm(df_dropped_channel["Participant"].unique()):
    df_participant = df_dropped_channel[df_dropped_channel["Participant"] == participant]

    for baseline in df_participant["Baseline"].unique():
        df_baseline = df_participant[df_participant["Baseline"] == baseline]

        # Init dataframes for participant psd data
        df_baseline_data_participant = pd.read_csv(f"./data/RQ/topo_data/filtered_participant/{participant}_{baseline}_baseline.csv", delimiter=";", names=columns)
        df_baseline_data_participant = df_baseline_data_participant.set_index(np.arange(1,65,1))
        df_program_data_participant = pd.read_csv(f"./data/RQ/topo_data/filtered_participant/{participant}_{baseline}_program.csv", delimiter=";", names=columns)
        df_program_data_participant = df_program_data_participant.set_index(np.arange(1,65,1))
        df_program_minus_baseline_data_participant = pd.read_csv(f"./data/RQ/topo_data/filtered_participant/{participant}_{baseline}_program_minus_baseline.csv", delimiter=";", names=columns)
        df_program_minus_baseline_data_participant = df_program_minus_baseline_data_participant.set_index(np.arange(1,65,1))

        for index, row in df_baseline.iterrows():
            if row["Type"] == "Baseline":
                df_baseline_data_participant.loc[row["Channel"], columns] = np.nan
                df_program_minus_baseline_data_participant.loc[row["Channel"], columns] = np.nan
            else:
                df_program_data_participant.loc[row["Channel"], columns] = np.nan
                df_program_minus_baseline_data_participant.loc[row["Channel"], columns] = np.nan

        # Save baseline psd data for participant
        baseline_data_participant = df_baseline_data_participant.to_numpy()
        np.savetxt(f"./data/RQ/topo_data/filtered_participant/{participant}_{baseline}_baseline.csv", baseline_data_participant, delimiter=";")

        # Save program psd data for participant
        program_data_participant = df_program_data_participant.to_numpy()
        np.savetxt(f"./data/RQ/topo_data/filtered_participant/{participant}_{baseline}_program.csv", program_data_participant, delimiter=";")

        # Save program minus baseline psd data for participant
        program_minus_baseline_data_participant = df_program_minus_baseline_data_participant.to_numpy()
        np.savetxt(f"./data/RQ/topo_data/filtered_participant/{participant}_{baseline}_program_minus_baseline.csv", program_minus_baseline_data_participant, delimiter=";")

#### Merge PSDS per Baseline

In [ ]:
# Create folder if not exists
if not os.path.exists("./data/RQ/topo_data/all"):
    os.makedirs("./data/RQ/topo_data/all")

for baseline in tqdm(df_topo["Baseline"].unique()):
    df_baseline = df_topo[df_topo["Baseline"] == baseline]

    # Init dataframes for baseline psd data
    df_baseline_data_all = pd.DataFrame(columns=columns_with_channel)
    df_program_data_all = pd.DataFrame(columns=columns_with_channel)
    df_program_minus_baseline_data_all = pd.DataFrame(columns=columns_with_channel)

    for participant in tqdm(df_baseline["Participant"].unique()):
        # Load psd data
        baseline_psds = np.loadtxt(f"./data/RQ/topo_data/filtered_participant/{participant}_{baseline}_baseline.csv", delimiter=";")
        program_psds = np.loadtxt(f"./data/RQ/topo_data/filtered_participant/{participant}_{baseline}_program.csv", delimiter=";")
        program_minus_baseline_psds = np.loadtxt(f"./data/RQ/topo_data/filtered_participant/{participant}_{baseline}_program_minus_baseline.csv", delimiter=";")

        channel_number = 1
        for channel in baseline_psds:
            # Concat channel number and baseline psd data for channel
            baseline_data = [channel_number]
            baseline_data.extend(channel)

            # Add channel psd data to dataframes
            df_baseline_data_all = pd.concat([df_baseline_data_all, pd.DataFrame(data=[baseline_data], columns=columns_with_channel)],ignore_index=True)
            channel_number += 1

        channel_number = 1
        for channel in program_psds:
            # Concat channel number and program psd data for channel
            program_data = [channel_number]
            program_data.extend(channel)

            # Add channel psd data to dataframes
            df_program_data_all = pd.concat([df_program_data_all, pd.DataFrame(data=[program_data], columns=columns_with_channel)],ignore_index=True)
            channel_number += 1

        channel_number = 1
        for channel in program_minus_baseline_psds:
            # Concat channel number and program psd data for channel
            program_minus_baseline_data = [channel_number]
            program_minus_baseline_data.extend(channel)

            # Add channel psd data to dataframes
            df_program_minus_baseline_data_all = pd.concat([df_program_minus_baseline_data_all, pd.DataFrame(data=[program_minus_baseline_data], columns=columns_with_channel)],ignore_index=True)
            channel_number += 1

    # Save baseline psd data for all
    df_baseline_data_all = df_baseline_data_all.groupby('Channel')[columns_with_channel[1:]].agg('mean')
    baseline_data_all = df_baseline_data_all.to_numpy()
    np.savetxt(f"./data/RQ/topo_data/all/{baseline}_baseline.csv", baseline_data_all, delimiter=";")

    # Save program psd data for all
    df_program_data_all = df_program_data_all.groupby('Channel')[columns_with_channel[1:]].agg('mean')
    program_data_all = df_program_data_all.to_numpy()
    np.savetxt(f"./data/RQ/topo_data/all/{baseline}_program.csv", program_data_all, delimiter=";")

    # Save program minus baseline psd data for all
    df_program_minus_baseline_data_all = df_program_minus_baseline_data_all.groupby('Channel')[columns_with_channel[1:]].agg('mean')
    program_minus_baseline_data_all = df_program_minus_baseline_data_all.to_numpy()
    np.savetxt(f"./data/RQ/topo_data/all/{baseline}_program_minus_baseline.csv", program_minus_baseline_data_all, delimiter=";")

#### Plot topoplots per baseline

In [ ]:
# Create folder if not exists
if not os.path.exists("./data/RQ/plots/topoplots/total/raw"):
    os.makedirs("./data/RQ/plots/topoplots/total/raw")
if not os.path.exists("./data/RQ/plots/topoplots/total/program_with_baseline"):
    os.makedirs("./data/RQ/plots/topoplots/total/program_with_baseline")

# Load info and freq data
epochs_data = mne.read_epochs(f"./data/RQ/topo_data/info-epo.fif", preload=True, verbose="ERROR")
info_data = epochs_data.info
freq_data = np.loadtxt(f"./data/RQ/topo_data/freq.csv", delimiter=";")

# Get channel data and unit
ch_type = mne.channels._get_ch_type(epochs_data, None)
units = mne.defaults._handle_default('units', None)
unit = units[ch_type]

# Init total figures per participant
all_topo_baseline_fig, all_topo_baseline_axs = plt.subplots(5, 4, sharex='all', sharey='all',figsize=(15,12))
all_topo_program_fig, all_topo_program_axs = plt.subplots(5, 4, sharex='all', sharey='all',figsize=(15,12))
all_topo_program_with_baseline_fig, all_topo_program_with_baseline_axs = plt.subplots(5, 4, sharex='all', sharey='all',figsize=(15,12))

all_topo_paper_fig, all_topo_paper_axs = plt.subplots(3, 4, sharex='all', sharey='all',figsize=(16,9))

# Row label
rows = ['A', 'B', 'C']
for ax, row in zip(all_topo_paper_axs[:,0], rows):
    ax.annotate(row, xy=(0, 0.5), xytext=(-ax.yaxis.labelpad - 5, 0), xycoords=ax.yaxis.label, textcoords='offset points', fontsize=20, ha='right', va='center')

count = 0

for baseline in tqdm(sorted(df_topo["Baseline"].unique())):

    # Get axs from total figures
    axs_all_baseline = [all_topo_baseline_axs[0,count%4], all_topo_baseline_axs[1,count%4], all_topo_baseline_axs[2,count%4], all_topo_baseline_axs[3,count%4], all_topo_baseline_axs[4,count%4]]
    axs_all_program = [all_topo_program_axs[0,count%4], all_topo_program_axs[1,count%4], all_topo_program_axs[2,count%4], all_topo_program_axs[3,count%4], all_topo_program_axs[4,count%4]]
    axs_all_program_with_baseline = [all_topo_program_with_baseline_axs[0,count%4], all_topo_program_with_baseline_axs[1,count%4], all_topo_program_with_baseline_axs[2,count%4], all_topo_program_with_baseline_axs[3,count%4], all_topo_program_with_baseline_axs[4,count%4]]
    axs_paper_without_baseline = [all_topo_paper_axs[0,count%4]]
    axs_paper_with_baseline = [all_topo_paper_axs[1,count%4]]
    axs_paper_with_baseline_theta = [all_topo_paper_axs[2,count%4]]

    # Load baseline psds
    psd_baseline_average = np.loadtxt(f"./data/RQ/topo_data/all/{baseline}_baseline.csv", delimiter=";")

    # Create total topoplot
    mne.viz.topomap.plot_psds_topomap(psd_baseline_average, freq_data, info_data, bands=[(4, 8, f'{translate(baseline)} Theta'), (8, 12, f'{translate(baseline)} Alpha'), (12, 30, f'{translate(baseline)} Beta'), (30, 45, f'{translate(baseline)} Gamma'), (0, 45, f'{translate(baseline)} All')], show=False, axes=axs_all_baseline, cmap="jet", dB=False, unit=unit, vlim=(0,25))

    # Init figure for single topoplot
    fig = plt.figure(tight_layout=True, figsize=(15, 3))

    # Create subfigures for single topoplot
    subfigure = fig.subfigures(nrows=1, ncols=1)
    axs = subfigure.subplots(nrows=1, ncols=5)

    # Create single topoplot
    mne.viz.topomap.plot_psds_topomap(psd_baseline_average, freq_data, info_data, bands=[(4, 8, f'Theta'), (8, 12, f'Alpha'), (12, 30, f'Beta'), (30, 45, f'Gamma'), (0, 45, f'All')], show=False, axes=axs, cmap="jet", dB=False, unit=unit, vlim=(0,25))

    # Save single topoplot
    plt.savefig(f"./data/RQ/plots/topoplots/total/raw/{translate(baseline)}_baseline_topo.pdf", bbox_inches='tight', pad_inches=0)
    plt.close()

    # Load program psds
    psd_program_average = np.loadtxt(f"./data/RQ/topo_data/all/{baseline}_program.csv", delimiter=";")

    # Create total topoplot
    mne.viz.topomap.plot_psds_topomap(psd_program_average, freq_data, info_data, bands=[(4, 8, f'$ProgCompr_{{{translate(baseline)}}}$ Theta'), (8, 12, f'$ProgCompr_{{{translate(baseline)}}}$ Alpha'), (12, 30, f'$ProgCompr_{{{translate(baseline)}}}$ Beta'), (30, 45, f'$ProgCompr_{{{translate(baseline)}}}$ Gamma'), (0, 45, f'$ProgCompr_{{{translate(baseline)}}}$ All')], show=False, axes=axs_all_program, cmap="jet", dB=False, unit=unit, vlim=(0,25))

    # Create paper topoplot
    mne.viz.topomap.plot_psds_topomap(psd_program_average, freq_data, info_data, bands=[(0, 45, f'$ProgCompr_{{{translate(baseline)}}}$')], show=False, axes=axs_paper_without_baseline, cmap="jet", dB=False, unit=unit, vlim=(0,25))

    # Init figure for single topoplot
    fig = plt.figure(tight_layout=True, figsize=(15, 3))

    # Create subfigures for single topoplot
    subfigure = fig.subfigures(nrows=1, ncols=1)
    axs = subfigure.subplots(nrows=1, ncols=5)

    # Create single topoplot
    mne.viz.topomap.plot_psds_topomap(psd_program_average, freq_data, info_data, bands=[(4, 8, f'Theta'), (8, 12, f'Alpha'), (12, 30, f'Beta'), (30, 45, f'Gamma'), (0, 45, f'All')], show=False, axes=axs, cmap="jet", dB=False, unit=unit, vlim=(0,25))

    # Save single topoplot
    plt.savefig(f"./data/RQ/plots/topoplots/total/raw/{translate(baseline)}_program_topo.pdf", bbox_inches='tight', pad_inches=0)
    plt.close()

    # Calculate program minus baseline psds
    psd_program_minus_baseline_average = np.loadtxt(f"./data/RQ/topo_data/all/{baseline}_program_minus_baseline.csv", delimiter=";")

    # Create total topoplot
    mne.viz.topomap.plot_psds_topomap(psd_program_minus_baseline_average, freq_data, info_data, bands=[(4, 8, f'$ProgCompr_{{{translate(baseline)}}}$ > {translate(baseline)} Theta'), (8, 12, f'$ProgCompr_{{{translate(baseline)}}}$ > {translate(baseline)} Alpha'), (12, 30, f'$ProgCompr_{{{translate(baseline)}}}$ > {translate(baseline)} Beta'), (30, 45, f'$ProgCompr_{{{translate(baseline)}}}$ > {translate(baseline)} Gamma'), (0, 45, f'$ProgCompr_{{{translate(baseline)}}}$ > {translate(baseline)} All')], cmap='jet', axes=axs_all_program_with_baseline, show=False, dB=False, unit=unit, vlim=(-3,3))

    # Create paper topoplot
    mne.viz.topomap.plot_psds_topomap(psd_program_minus_baseline_average, freq_data, info_data, bands=[(0, 45, f'$ProgCompr_{{{translate(baseline)}}}$ > {translate(baseline)}')], show=False, axes=axs_paper_with_baseline, cmap="jet", dB=False, unit=unit, vlim=(-3,3))
    mne.viz.topomap.plot_psds_topomap(psd_program_minus_baseline_average, freq_data, info_data, bands=[(4, 8, f'$ProgCompr_{{{translate(baseline)}}}$ > {translate(baseline)} Theta')], show=False, axes=axs_paper_with_baseline_theta, cmap="jet", dB=False, unit=unit, vlim=(-3,3))

    # Init figure for single topoplot
    fig = plt.figure(tight_layout=True, figsize=(15, 3))

    # Create subfigures for single topoplot
    subfigure = fig.subfigures(nrows=1, ncols=1)
    axs = subfigure.subplots(nrows=1, ncols=5)

    # Create single topoplot
    mne.viz.topomap.plot_psds_topomap(psd_program_minus_baseline_average, freq_data, info_data, bands=[(4, 8, f'Theta'), (8, 12, f'Alpha'), (12, 30, f'Beta'), (30, 45, f'Gamma'), (0, 45, f'All')], cmap='jet', axes=axs, show=False, dB=False, unit=unit, vlim=(-3,3))

    # Save single topoplot
    plt.savefig(f"./data/RQ/plots/topoplots/total/program_with_baseline/{translate(baseline)}_topo.pdf", bbox_inches='tight', pad_inches=0)
    plt.close()

    axs_paper_without_baseline[0].title.set_size(15)
    axs_paper_with_baseline[0].title.set_size(15)
    axs_paper_with_baseline_theta[0].title.set_size(15)


    count += 1

# Save total plots
plt.figure(all_topo_baseline_fig)
plt.savefig(f"./data/RQ/plots/topoplots/total/raw/all_baseline_topo.pdf", bbox_inches='tight', pad_inches=0)
plt.close()

plt.figure(all_topo_program_fig)
plt.savefig(f"./data/RQ/plots/topoplots/total/raw/all_program_topo.pdf", bbox_inches='tight', pad_inches=0)
plt.close()

plt.figure(all_topo_program_with_baseline_fig)
plt.savefig(f"./data/RQ/plots/topoplots/total/program_with_baseline/all_topo.pdf", bbox_inches='tight', pad_inches=0)
plt.close()

for ax in all_topo_paper_fig.get_axes():
    ax.xaxis.label.set_size(12)
    ax.yaxis.label.set_size(12)
    try:
        cb = ax.images[-1].colorbar
        ticklabs = cb.ax.get_yticklabels()
        cb.ax.set_yticklabels(ticklabs, fontsize=12)
    except:
        pass
plt.figure(all_topo_paper_fig)
plt.savefig(f"./data/RQ/plots/topoplots/total/topo_paper.pdf", bbox_inches='tight', pad_inches=0)
plt.close()